<a href="https://colab.research.google.com/github/GitToWork-2025/Hackathon/blob/main/UCL_Datathon_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

### Read the data


In [6]:
zip_path =r"/content/drive/MyDrive/ucl-datathon.zip"

### Extracting zip file and reading data




In [9]:
import zipfile
import os
import pandas as pd

extract_path = 'extracted_data'  # folder cted

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Optional: list the extracted files
extracted_files = os.listdir(extract_path)
print("Extracted files:", extracted_files)


Extracted files: ['train.json', 'sample_submission.csv', 'test_matchups.json']


In [10]:
sample_path =r'/content/extracted_data/sample_submission.csv'
sample_df =pd.read_csv(sample_path)
sample_df.head()

,id,season,predictions
0,0,2017-18,"{""round_of_16"": [{""team_1"": ""Real Madrid"", ""te..."
1,1,2018-19,"{""round_of_16"": [{""team_1"": ""Barcelona"", ""team..."
2,2,2019-20,"{""round_of_16"": [{""team_1"": ""Real Madrid"", ""te..."
3,3,2020-21,"{""round_of_16"": [{""team_1"": ""Barcelona"", ""team..."
4,4,2021-22,"{""round_of_16"": [{""team_1"": ""Real Madrid"", ""te..."


In [11]:
from google.colab import files
sample_df.to_csv("sample_sub.csv", index = False)
files.download("sample_sub.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import json

json_path = os.path.join(extract_path, 'train.json')
json_test = os.path.join(extract_path, 'test_matchups.json')
# Read the JSON file
with open(json_path, 'r') as f:
    ucl_data = json.load(f)

with open(json_test, 'r') as f:
    ucl_test = json.load(f)
# Preview the structure
print(type(ucl_data))       # Could be a list or dict
print(len(ucl_data))        # Number of entries
print(len(ucl_test))


<class 'dict'>
13
7


In [ ]:
print(ucl_test.keys())
print(ucl_test['2022-23'].keys())
print(ucl_test['2022-23'].keys)

dict_keys(['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24'])
dict_keys(['round_of_16_matchups'])
<built-in method keys of dict object at 0x7ec53c2db380>


In [13]:
import pandas as pd

# Flatten the nested dictionary
matches = []
for season, stages in ucl_data.items():
    for stage, games in stages.items():
        for match in games:
            matches.append({
                'season': season,
                'stage': stage,
                'date': match['date'],
                'team_1': match['team_1'],
                'team_2': match['team_2'],
                'winner': match['winner']
            })

plays = []
for season, stages in ucl_test.items():
    for stage, games in stages.items():
        for match in games:
            plays.append({
                'season': season,
                'stage': stage,
                'team_1': match['team_1'],
                'team_2': match['team_2'],
            })
# Create DataFrame
train_df= pd.DataFrame(matches)
test_df = pd.DataFrame(plays)



In [14]:
test_df.head()


,season,stage,team_1,team_2
0,2017-18,round_of_16_matchups,Juventus,Tottenham Hotspur
1,2017-18,round_of_16_matchups,Basel,Manchester City
2,2017-18,round_of_16_matchups,Porto,Liverpool
3,2017-18,round_of_16_matchups,Sevilla,Manchester United
4,2017-18,round_of_16_matchups,Real Madrid,Paris Saint-Germain


In [ ]:
train_df.head()

,season,stage,date,team_1,team_2,winner
0,2004-05,round_of_16,22/02/2005,Real Madrid,Juventus,Juventus
1,2004-05,round_of_16,22/02/2005,Liverpool,Bayer Leverkusen,Liverpool
2,2004-05,round_of_16,22/02/2005,PSV Eindhoven,Monaco,PSV Eindhoven
3,2004-05,round_of_16,22/02/2005,Bayern Munich,Arsenal,Bayern Munich
4,2004-05,round_of_16,23/02/2005,Barcelona,Chelsea,Chelsea


In [15]:
train_df.isnull().sum()

,0
season,0
stage,0
date,0
team_1,0
team_2,0
winner,0


In [16]:
# Recalculate win rates
team_win_counts = train_df['winner'].value_counts()
team_match_counts = pd.concat([train_df['team_1'], train_df['team_2']]).value_counts()
team_win_rate = (team_win_counts / team_match_counts).fillna(0)

train_df['team_1_win_rate'] = train_df['team_1'].map(team_win_rate)
train_df['team_2_win_rate'] = train_df['team_2'].map(team_win_rate)

# Head-to-head wins for team_1 over team_2
def head_to_head_win(row):
    t1 = row['team_1']
    t2 = row['team_2']
    mask = (train_df['team_1'] == t1) & (train_df['team_2'] == t2) & (train_df['winner'] == t1)
    return train_df[mask].shape[0]

train_df['head_to_head_t1'] = train_df.apply(head_to_head_win, axis=1)

# Knockout stage experience
knockout_stages = ['quarter_finals','semi_finals', 'final']
knockout_exp = train_df[train_df['stage'].isin(knockout_stages)]
team_knockout_exp = pd.concat([knockout_exp['team_1'], knockout_exp['team_2']]).value_counts()

train_df['team_1_knockout_exp'] = train_df['team_1'].map(team_knockout_exp).fillna(0)
train_df['team_2_knockout_exp'] = train_df['team_2'].map(team_knockout_exp).fillna(0)

# Show enriched features
train_df[['team_1', 'team_2', 'team_1_win_rate', 'team_2_win_rate', 'head_to_head_t1', 'team_1_knockout_exp', 'team_2_knockout_exp']].tail()
train_df.head()


,season,stage,date,team_1,team_2,winner,team_1_win_rate,team_2_win_rate,head_to_head_t1,team_1_knockout_exp,team_2_knockout_exp
0,2004-05,round_of_16,22/02/2005,Real Madrid,Juventus,Juventus,0.666667,0.562500,0,17.0,9.0
1,2004-05,round_of_16,22/02/2005,Liverpool,Bayer Leverkusen,Liverpool,0.714286,0.000000,1,9.0,0.0
2,2004-05,round_of_16,22/02/2005,PSV Eindhoven,Monaco,PSV Eindhoven,0.428571,0.500000,1,3.0,3.0
3,2004-05,round_of_16,22/02/2005,Bayern Munich,Arsenal,Bayern Munich,0.677419,0.350000,2,19.0,7.0
4,2004-05,round_of_16,23/02/2005,Barcelona,Chelsea,Chelsea,0.750000,0.653846,1,23.0,15.0


### Data Addition


In [17]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd

# Specify the URL for the desired season
url = 'https://fbref.com/en/comps/8/2023-2024/2023-2024-Champions-League-Stats'

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract all comments from the page
comments = soup.find_all(string=lambda text: isinstance(text, Comment))
print(comments)
# Iterate through comments to find tables
for comment in comments:
    comment_soup = BeautifulSoup(comment, 'html.parser')
    tables = comment_soup.find_all('table')
    print(table)
    for table in tables:
        # Check if the table has the desired ID
        if table.get('id') == 'league_table':
            dff = pd.read_html(str(table))[0]
            print(dff.head())


[]


In [18]:
train_df.to_csv("data_with_additional_features.csv", index=False)


In [19]:
from google.colab import files
files.download("data_with_additional_features.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Encoding the data


In [21]:
all_teams = pd.concat([train_df['team_1'], train_df['team_2']]).unique()
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(all_teams)
train_df['team_1_encoded'] = le.transform(train_df['team_1'])
train_df['team_2_encoded'] = le.transform(train_df['team_2'])
train_df['winner_encoded'] = le.transform(train_df['winner'])
train_df['stage_encoded'] = le.fit_transform(train_df['stage'])
train_df['season_encoded'] = le.fit_transform(train_df['season'])
train_df.head()

,season,stage,date,team_1,team_2,winner,team_1_win_rate,team_2_win_rate,head_to_head_t1,team_1_knockout_exp,team_2_knockout_exp,team_1_encoded,team_2_encoded,winner_encoded,stage_encoded,season_encoded
0,2004-05,round_of_16,22/02/2005,Real Madrid,Juventus,Juventus,0.666667,0.562500,0,17.0,9.0,39,21,21,2,0
1,2004-05,round_of_16,22/02/2005,Liverpool,Bayer Leverkusen,Liverpool,0.714286,0.000000,1,9.0,0.0,24,6,24,2,0
2,2004-05,round_of_16,22/02/2005,PSV Eindhoven,Monaco,PSV Eindhoven,0.428571,0.500000,1,3.0,3.0,34,31,34,2,0
3,2004-05,round_of_16,22/02/2005,Bayern Munich,Arsenal,Bayern Munich,0.677419,0.350000,2,19.0,7.0,7,2,7,2,0
4,2004-05,round_of_16,23/02/2005,Barcelona,Chelsea,Chelsea,0.750000,0.653846,1,23.0,15.0,4,13,13,2,0


### Training baseline model


In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load data
train_df = pd.read_csv("data_with_additional_features.csv")

le_team = LabelEncoder()
all_teams = pd.concat([train_df['team_1'], train_df['team_2']])
le_team.fit(all_teams)

# Encode teams and winner
train_df['team_1_encoded'] = le_team.transform(train_df['team_1'])
train_df['team_2_encoded'] = le_team.transform(train_df['team_2'])
train_df['winner_encoded'] = le_team.transform(train_df['winner'])


# Features and target
features = [
    'team_1_encoded', 'team_2_encoded',
    'team_1_win_rate', 'team_2_win_rate',
    'head_to_head_t1', 'team_1_knockout_exp', 'team_2_knockout_exp'
]
X = train_df[features]
y = train_df['winner_encoded']

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest baseline model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5385


In [28]:
test_df.head()
train_df.head()

,season,stage,date,team_1,team_2,winner,team_1_win_rate,team_2_win_rate,head_to_head_t1,team_1_knockout_exp,team_2_knockout_exp,team_1_encoded,team_2_encoded,winner_encoded
0,2004-05,round_of_16,22/02/2005,Real Madrid,Juventus,Juventus,0.666667,0.562500,0,17.0,9.0,39,21,21
1,2004-05,round_of_16,22/02/2005,Liverpool,Bayer Leverkusen,Liverpool,0.714286,0.000000,1,9.0,0.0,24,6,24
2,2004-05,round_of_16,22/02/2005,PSV Eindhoven,Monaco,PSV Eindhoven,0.428571,0.500000,1,3.0,3.0,34,31,34
3,2004-05,round_of_16,22/02/2005,Bayern Munich,Arsenal,Bayern Munich,0.677419,0.350000,2,19.0,7.0,7,2,7
4,2004-05,round_of_16,23/02/2005,Barcelona,Chelsea,Chelsea,0.750000,0.653846,1,23.0,15.0,4,13,13


In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Fit team encoder on both team_1 and team_2 from training set
le_team = LabelEncoder()
all_teams = pd.concat([test_df['team_1'], test_df['team_2']])
le_team.fit(all_teams)

test_df['team_1_encoded'] = le_team.transform(test_df['team_1'])
test_df['team_2_encoded'] = le_team.transform(test_df['team_2'])

# Win rate feature from training set
team_win_counts = train_df['winner'].value_counts()
team_total_matches = pd.concat([train_df['team_1'], train_df['team_2']]).value_counts()
team_win_rate = (team_win_counts / team_total_matches).fillna(0)

test_df['team_1_win_rate'] = test_df['team_1'].map(team_win_rate).fillna(0)
test_df['team_2_win_rate'] = test_df['team_2'].map(team_win_rate).fillna(0)

# Head-to-head wins (team_1 vs team_2)
def head_to_head_t1(row):
    return train_df[
        (train_df['team_1'] == row['team_1']) &
        (train_df['team_2'] == row['team_2']) &
        (train_df['winner'] == row['team_1'])
    ].shape[0]

test_df['head_to_head_t1'] = test_df.apply(head_to_head_t1, axis=1)

# Knockout experience (quarterfinals and beyond)
knockout_stages = ['quarter_finals', 'semi_finals', 'final']
knockout_df = train_df[train_df['stage'].isin(knockout_stages)]
team_knockout_exp = pd.concat([knockout_df['team_1'], knockout_df['team_2']]).value_counts()

test_df['team_1_knockout_exp'] = test_df['team_1'].map(team_knockout_exp).fillna(0)
test_df['team_2_knockout_exp'] = test_df['team_2'].map(team_knockout_exp).fillna(0)

# Final prepared test features
X_test = test_df[[
    'team_1_encoded', 'team_2_encoded',
    'team_1_win_rate', 'team_2_win_rate',
    'head_to_head_t1', 'team_1_knockout_exp', 'team_2_knockout_exp'
]]

X_test.head()


,team_1_encoded,team_2_encoded,team_1_win_rate,team_2_win_rate,head_to_head_t1,team_1_knockout_exp,team_2_knockout_exp
0,17,37,0.562500,0.500000,0,9.0,1.0
1,6,22,0.000000,0.333333,0,0.0,2.0
2,27,20,0.200000,0.714286,0,2.0,9.0
3,34,23,0.000000,0.666667,0,0.0,13.0
4,29,26,0.666667,0.333333,0,17.0,4.0


In [66]:
y_pred_encoded1 = model.predict(X_test)
y_pred_encoded1
test_df['predicted_winner'] = le_team.inverse_transform(y_pred_encoded1)
#Need to use le_team of train_df as not all teams are in test_df



ValueError: y contains previously unseen labels: [46]

In [34]:
test_df.shape

(56, 11)

In [46]:
test_df.to_csv("test_1.csv", index=False)
from google.colab import files
files.download("test_1.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
test_df.head()

,season,stage,team_1,team_2,team_1_encoded,team_2_encoded,team_1_win_rate,team_2_win_rate,head_to_head_t1,team_1_knockout_exp,team_2_knockout_exp
0,2017-18,round_of_16_matchups,Juventus,Tottenham Hotspur,17,37,0.562500,0.500000,0,9.0,1.0
1,2017-18,round_of_16_matchups,Basel,Manchester City,6,22,0.000000,0.333333,0,0.0,2.0
2,2017-18,round_of_16_matchups,Porto,Liverpool,27,20,0.200000,0.714286,0,2.0,9.0
3,2017-18,round_of_16_matchups,Sevilla,Manchester United,34,23,0.000000,0.666667,0,0.0,13.0
4,2017-18,round_of_16_matchups,Real Madrid,Paris Saint-Germain,29,26,0.666667,0.333333,0,17.0,4.0


In [49]:
test_df['stage'] = test_df['stage'].str.replace('_matchups', '', regex=False)

def group_predictions(test_df):
    matches = []
    for _, row in test_df.iterrows():
        match_dict = {
            "team_1": row['team_1'],
            "team_2": row['team_2'],
           # "winner": row['predicted_winner']
        }
        matches.append(match_dict)
    return matches
preds =group_predictions(test_df)
print(len(preds))
print(preds)



56
[{'team_1': 'Juventus', 'team_2': 'Tottenham Hotspur'}, {'team_1': 'Basel', 'team_2': 'Manchester City'}, {'team_1': 'Porto', 'team_2': 'Liverpool'}, {'team_1': 'Sevilla', 'team_2': 'Manchester United'}, {'team_1': 'Real Madrid', 'team_2': 'Paris Saint-Germain'}, {'team_1': 'Shakhtar Donetsk', 'team_2': 'Roma'}, {'team_1': 'Chelsea', 'team_2': 'Barcelona'}, {'team_1': 'Bayern Munich', 'team_2': 'Besiktas'}, {'team_1': 'Roma', 'team_2': 'Porto'}, {'team_1': 'Manchester United', 'team_2': 'Paris Saint-Germain'}, {'team_1': 'Tottenham Hotspur', 'team_2': 'Borussia Dortmund'}, {'team_1': 'Ajax', 'team_2': 'Real Madrid'}, {'team_1': 'Lyon', 'team_2': 'Barcelona'}, {'team_1': 'Liverpool', 'team_2': 'Bayern Munich'}, {'team_1': 'Atletico Madrid', 'team_2': 'Juventus'}, {'team_1': 'Schalke 04', 'team_2': 'Manchester City'}, {'team_1': 'Borussia Dortmund', 'team_2': 'Paris Saint-Germain'}, {'team_1': 'Real Madrid', 'team_2': 'Manchester City'}, {'team_1': 'Atalanta', 'team_2': 'Valencia'}, {

In [50]:
l = []
for i in range(7):
  dic={}
  dic["round_of_16"]=preds[i:i+7]
  l.append(dic)
print(l)

[{'round_of_16': [{'team_1': 'Juventus', 'team_2': 'Tottenham Hotspur'}, {'team_1': 'Basel', 'team_2': 'Manchester City'}, {'team_1': 'Porto', 'team_2': 'Liverpool'}, {'team_1': 'Sevilla', 'team_2': 'Manchester United'}, {'team_1': 'Real Madrid', 'team_2': 'Paris Saint-Germain'}, {'team_1': 'Shakhtar Donetsk', 'team_2': 'Roma'}, {'team_1': 'Chelsea', 'team_2': 'Barcelona'}]}, {'round_of_16': [{'team_1': 'Basel', 'team_2': 'Manchester City'}, {'team_1': 'Porto', 'team_2': 'Liverpool'}, {'team_1': 'Sevilla', 'team_2': 'Manchester United'}, {'team_1': 'Real Madrid', 'team_2': 'Paris Saint-Germain'}, {'team_1': 'Shakhtar Donetsk', 'team_2': 'Roma'}, {'team_1': 'Chelsea', 'team_2': 'Barcelona'}, {'team_1': 'Bayern Munich', 'team_2': 'Besiktas'}]}, {'round_of_16': [{'team_1': 'Porto', 'team_2': 'Liverpool'}, {'team_1': 'Sevilla', 'team_2': 'Manchester United'}, {'team_1': 'Real Madrid', 'team_2': 'Paris Saint-Germain'}, {'team_1': 'Shakhtar Donetsk', 'team_2': 'Roma'}, {'team_1': 'Chelsea', 

In [57]:
data = {
    'id' : [0,1,2,3,4,5,6],
    'season' : ['2017-18','2018-19','2019-20','2020-21','2021-22','2022-23','2023-24'],
    'predictions' : l}

sub_df = pd.DataFrame(data)
sub_df.head()

,id,season,predictions
0,0,2017-18,"{'round_of_16': [{'team_1': 'Juventus', 'team_..."
1,1,2018-19,"{'round_of_16': [{'team_1': 'Basel', 'team_2':..."
2,2,2019-20,"{'round_of_16': [{'team_1': 'Porto', 'team_2':..."
3,3,2020-21,"{'round_of_16': [{'team_1': 'Sevilla', 'team_2..."
4,4,2021-22,"{'round_of_16': [{'team_1': 'Real Madrid', 'te..."


In [58]:
sub_df.to_csv("sub_1.csv", index=False)
from google.colab import files
files.download("sub_1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
import pandas as pd
import json

# Step 1: Load the CSV file
df = pd.read_csv('sub_1.csv')  # Make sure the file is in the same directory or provide full path

# Step 2: Fix the 'predictions' column
def convert_json_string(s):
    try:
        # Convert the string with single quotes to a Python dict
        obj = eval(s)
        # Convert it back to a proper JSON string with double quotes
        return json.dumps(obj)
    except:
        return s  # If there's an error, keep the original

df['predictions'] = df['predictions'].apply(convert_json_string)

# Step 3: Save to a new CSV file
df.to_csv('sub_1_json_ready.csv', index=False)


In [60]:

from google.colab import files
files.download("sub_1_json_ready.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [61]:
test_df.tail()

,season,stage,team_1,team_2,team_1_encoded,team_2_encoded,team_1_win_rate,team_2_win_rate,head_to_head_t1,team_1_knockout_exp,team_2_knockout_exp
51,2023-24,round_of_16,Paris Saint-Germain,Real Sociedad,26,30,0.333333,0.000000,0,4.0,0.0
52,2023-24,round_of_16,Inter Milan,Atletico Madrid,16,4,0.000000,0.642857,0,0.0,9.0
53,2023-24,round_of_16,PSV Eindhoven,Borussia Dortmund,25,10,0.428571,0.555556,0,3.0,5.0
54,2023-24,round_of_16,Porto,Arsenal,27,2,0.200000,0.350000,0,2.0,7.0
55,2023-24,round_of_16,Napoli,Barcelona,24,5,0.000000,0.750000,0,0.0,23.0
